In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
## extra imports to set GPU options
import tensorflow as tf
from keras import backend as k

###################################
# TensorFlow wizardry
config = tf.ConfigProto()

# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 1 # 0.5 for Half

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))


Using TensorFlow backend.


In [1]:
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.utils import plot_model

import tensorflow as tf
import keras
import seaborn as sn
import pandas as pd
import numpy as np
import linecache
import sys
import h5py
import itertools
import matplotlib.pyplot as plt
import os
import pickle
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
tf.__version__

'1.9.0'

In [5]:
train_path = '/data/danish/Data/Malimg-Gray-25-Families/Malimg_Grayscale_train/'
test_path  = '/data/danish/Data/Malimg-Gray-25-Families/Malimg_Grayscale_test/'

In [6]:
datagen_train = ImageDataGenerator(
      rescale=1./255#,
      #rotation_range=0.1,#180,
      #width_shift_range=0.1,
      #height_shift_range=0.1,
      #shear_range=0.1,
      #zoom_range=0.1,#[0.9, 1.5],
      #horizontal_flip=True,
      #vertical_flip=True,
      #fill_mode='nearest'
                )

datagen_test = ImageDataGenerator(rescale=1./255)


In [7]:
if False:
    save_to_dir='augmented_images/'
else:
    save_to_dir = None

In [8]:
input_shape=(224,224)
batch_size=32
save_fn='Deep_Learning_Model_for_Malimg_size_GrayScale='+str(input_shape[:])
save_fn

'Deep_Learning_Model_for_Malimg_size_GrayScale=(224, 224)'

In [9]:
generator_train = datagen_train.flow_from_directory(directory=train_path,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=42)


steps_train = generator_train.n/ batch_size
print("Steps for Training",steps_train)

cls_train = generator_train.classes
print(cls_train)
cls_names_train = list(generator_train.class_indices.keys())
print(cls_names_train)
num_cls_train = len(cls_names_train)
print("Number of Classes in Training Data",num_cls_train)

Found 6527 images belonging to 25 classes.
Steps for Training 203.96875
[ 0  0  0 ..., 24 24 24]
['Adialer.C', 'Agent.FYI', 'Allaple.A', 'Allaple.L', 'Alueron.gen!J', 'Autorun.K', 'C2LOP.P', 'C2LOP.gen!g', 'Dialplatform.B', 'Dontovo.A', 'Fakerean', 'Instantaccess', 'Lolyda.AA1', 'Lolyda.AA2', 'Lolyda.AA3', 'Lolyda.AT', 'Malex.gen!J', 'ObfuscatorAD', 'Rbot!gen', 'Skintrim.N', 'Swizzorgen!E', 'Swizzorgen!I', 'VB.AT', 'Wintrim.BX', 'Yuner.A']
Number of Classes in Training Data 25


In [10]:
#cls_names_train
class_labels = ['AdialerC','AgentFYI','AllapleA','AllapleL','Aluerongen!J','AutorunK','C2LOPP','C2LOPgen!g','DialplatformB','DontovoA',
 'Fakerean','Instantaccess','LolydaAA1','LolydaAA2','LolydaAA3','LolydaAT','Malexgen!J','ObfuscatorAD','Rbot!gen','SkintrimN',
 'Swizzorgen!E','Swizzorgen!I','VBAT','WintrimBX','YunerA']
class_labels

['AdialerC',
 'AgentFYI',
 'AllapleA',
 'AllapleL',
 'Aluerongen!J',
 'AutorunK',
 'C2LOPP',
 'C2LOPgen!g',
 'DialplatformB',
 'DontovoA',
 'Fakerean',
 'Instantaccess',
 'LolydaAA1',
 'LolydaAA2',
 'LolydaAA3',
 'LolydaAT',
 'Malexgen!J',
 'ObfuscatorAD',
 'Rbot!gen',
 'SkintrimN',
 'Swizzorgen!E',
 'Swizzorgen!I',
 'VBAT',
 'WintrimBX',
 'YunerA']

In [11]:
generator_test = datagen_test.flow_from_directory(directory=test_path,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  seed=42)

steps_test = generator_test.n / batch_size
steps_test
print("Steps for Test",steps_test)

cls_test = generator_test.classes
print(cls_test)
cls_names_test = list(generator_test.class_indices.keys())
print(cls_names_test)
num_cls_test = len(cls_names_test)
print("Number of Classes in Test Data",num_cls_test)

Found 2812 images belonging to 25 classes.
Steps for Test 87.875
[ 0  0  0 ..., 24 24 24]
['Adialer.C', 'Agent.FYI', 'Allaple.A', 'Allaple.L', 'Alueron.gen!J', 'Autorun.K', 'C2LOP.P', 'C2LOP.gen!g', 'Dialplatform.B', 'Dontovo.A', 'Fakerean', 'Instantaccess', 'Lolyda.AA1', 'Lolyda.AA2', 'Lolyda.AA3', 'Lolyda.AT', 'Malex.gen!J', 'ObfuscatorAD', 'Rbot!gen', 'Skintrim.N', 'Swizzorgen!E', 'Swizzorgen!I', 'VB.AT', 'Wintrim.BX', 'Yuner.A']
Number of Classes in Test Data 25


In [12]:
class_dictionary_test = generator_test.class_indices
class_dictionary_test

{'Adialer.C': 0,
 'Agent.FYI': 1,
 'Allaple.A': 2,
 'Allaple.L': 3,
 'Alueron.gen!J': 4,
 'Autorun.K': 5,
 'C2LOP.P': 6,
 'C2LOP.gen!g': 7,
 'Dialplatform.B': 8,
 'Dontovo.A': 9,
 'Fakerean': 10,
 'Instantaccess': 11,
 'Lolyda.AA1': 12,
 'Lolyda.AA2': 13,
 'Lolyda.AA3': 14,
 'Lolyda.AT': 15,
 'Malex.gen!J': 16,
 'ObfuscatorAD': 17,
 'Rbot!gen': 18,
 'Skintrim.N': 19,
 'Swizzorgen!E': 20,
 'Swizzorgen!I': 21,
 'VB.AT': 22,
 'Wintrim.BX': 23,
 'Yuner.A': 24}

In [13]:
model_ = keras.applications.vgg16.VGG16()#include_top=True, weights=None, input_tensor=None, input_shape=(224,224,1), classes=num_cls_train)

In [14]:
transfer_layer = model_.get_layer('fc2')

In [15]:
transfer_layer.output

<tf.Tensor 'fc2/Relu:0' shape=(?, 4096) dtype=float32>

In [16]:
conv_model = Model(inputs=model_.input,
                   outputs=transfer_layer.output)

In [17]:
conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [18]:
# Start a new Keras Sequential model.
model = Sequential()

# Add the convolutional part of the VGG16 model from above.
model.add(conv_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
#model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
#model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
#model.add(Dropout(0.5))

# Add the final layer for the actual classification.
model.add(Dense(num_cls_train, activation='softmax'))

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 4096)              134260544 
_________________________________________________________________
dense_1 (Dense)              (None, 25)                102425    
Total params: 134,362,969
Trainable params: 134,362,969
Non-trainable params: 0
_________________________________________________________________


In [20]:

#model.load_weights('Deep_Learning_Model_for_Malimg_size=(192, 192)_Weights.hdf5')
model.compile(Adam(lr=0.0001),loss='categorical_crossentropy', metrics=['accuracy'])


In [21]:
epochs =10
steps_per_epoch=steps_train

In [22]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import RemoteMonitor



history = model.fit_generator(generator=generator_train,
                                epochs=epochs,
                                steps_per_epoch=steps_per_epoch,
                                validation_data=generator_test,
                                validation_steps=steps_test
                                )




Epoch 1/10
204/203 [==============================] - 89s 438ms/step - loss: 0.5495 - acc: 0.8627 - val_loss: 0.1295 - val_acc: 0.9595
Epoch 2/10
204/203 [==============================] - 75s 365ms/step - loss: 0.1247 - acc: 0.9595 - val_loss: 0.2309 - val_acc: 0.9470
Epoch 3/10
204/203 [==============================] - 74s 363ms/step - loss: 0.4317 - acc: 0.9064 - val_loss: 0.1460 - val_acc: 0.9563
Epoch 4/10
204/203 [==============================] - 74s 361ms/step - loss: 0.0944 - acc: 0.9691 - val_loss: 0.0931 - val_acc: 0.9708
Epoch 5/10
204/203 [==============================] - 74s 364ms/step - loss: 0.0693 - acc: 0.9764 - val_loss: 0.0840 - val_acc: 0.9730
Epoch 6/10
204/203 [==============================] - 74s 363ms/step - loss: 0.1548 - acc: 0.9611 - val_loss: 0.1194 - val_acc: 0.9609
Epoch 7/10
204/203 [==============================] - 75s 366ms/step - loss: 0.0943 - acc: 0.9692 - val_loss: 0.1537 - val_acc: 0.9591
Epoch 8/10
204/203 [==============================] - 7

In [2]:

# read Traning History back from the file
pkl_file = open("Original-VGG16-Grayscale-History.pkl", 'rb')
history = pickle.load(pkl_file)
pkl_file.close()

In [3]:
import pandas as pd
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history) 

csv_file = 'Original-VGG16-Grayscale-History.csv'
with open(csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [26]:
#scores = model.evaluate(X[test], Y[test], verbose=0)
#model.load_weights('Model_VGG16_Malimg_RGB_Trained_(229, 229)_Epochs=100_Model_Traning_Testing_Weights.hdf5')
scores = model.evaluate_generator(generator_test, verbose=1)# steps_test, max_queue_size=10, workers=1, use_multiprocessing=False,verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

88/88 [==============================] - 11s 127ms/step
acc: 97.30%


In [27]:
cls_pred = model.predict_generator(generator_test, verbose=1)#, steps_test, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

88/88 [==============================] - 12s 131ms/step


In [28]:
cls_pred = np.argmax(cls_pred, axis=1)

In [29]:
y_test = cls_test = generator_test.classes

from pycm import ConfusionMatrix
CM_IMCFN = ConfusionMatrix(y_test,cls_pred)
file = open('CM-Original-VGG16-Grayscale.pkl', 'wb')
pickle.dump(CM_IMCFN, file)
file.close()

In [30]:
# write Traning History to a file
file = open('CM-Original-VGG16-Grayscale.pkl', 'rb')
CM_IMCFN = pickle.load(file)
file.close()

In [31]:
CM_IMCFN.stat(summary=True)

Overall Statistics : 

ACC Macro                                                         0.99784
F1 Macro                                                          0.91817
Kappa                                                             0.96831
Overall ACC                                                       0.97297
PPV Macro                                                         None
SOA1(Landis & Koch)                                               Almost Perfect
TPR Macro                                                         0.92017
Zero-one Loss                                                     76

Class Statistics :

Classes                                                           0             1             10            11            12            13            14            15            16            17            18            19            2             20            21            22            23            24            3             4             5             6    

/home/danish/anaconda3/envs/tf/lib/python3.6/site-packages/pycm/pycm_obj.py:167: RuntimeWarning: The confusion matrix is a high dimension matrix and won't be demonstrated properly.
The save_csv method can be used to save the confusion matrix in csv format and have a better demonstration of it.
  warn(CLASS_NUMBER_WARNING, RuntimeWarning)


In [32]:
CM_IMCFN.save_csv("CM-Original-VGG16-Grayscale",address=True)

{'Status': True,
 'Message': '/data/danish/ImgProPython/TensorFlow-Tutorials/Computer-Networks-(CCF-B)/Original-VGG16-Grayscale/CM-Original-VGG16-Grayscale.csv'}

In [33]:
%%time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
cls_true = cls_test = generator_test.classes
accuracy = accuracy_score(cls_true, cls_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(cls_true, cls_pred, average=None)

print("All Classes")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

All Classes
Precision:  [ 1.          1.          0.99550056  1.          1.          0.          0.78
  0.91803279  1.          1.          1.          1.          0.94117647
  1.          1.          0.97959184  1.          1.          1.          1.
  0.76923077  0.70967742  1.          0.96774194  0.88235294]
Recall:  [ 1.          1.          1.          1.          1.          0.
  0.88636364  0.93333333  0.96296296  1.          0.99130435  1.          1.
  0.96428571  1.          1.          0.97560976  1.          0.97916667
  1.          0.76923077  0.55        0.99186992  1.          1.        ]
F1 score:  [ 1.          1.          0.99774521  1.          1.          0.
  0.82978723  0.92561983  0.98113208  1.          0.99563319  1.
  0.96969697  0.98181818  1.          0.98969072  0.98765432  1.
  0.98947368  1.          0.76923077  0.61971831  0.99591837  0.98360656
  0.9375    ]
CPU times: user 12.3 ms, sys: 33 µs, total: 12.3 ms
Wall time: 11.6 ms


/home/danish/anaconda3/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
%%time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
cls_true = cls_test = generator_test.classes
accuracy = accuracy_score(cls_true, cls_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(cls_true, cls_pred, average='micro')

print("Micro")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

Micro
Precision:  0.972972972973
Recall:  0.972972972973
F1 score:  0.972972972973
CPU times: user 4.49 ms, sys: 0 ns, total: 4.49 ms
Wall time: 6.41 ms


In [35]:
%%time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
cls_true = cls_test = generator_test.classes
accuracy = accuracy_score(cls_true, cls_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(cls_true, cls_pred, average='macro')

print("Macro")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

Macro
Precision:  0.917732188875
Recall:  0.920165084219
F1 score:  0.91816901691
CPU times: user 3.78 ms, sys: 0 ns, total: 3.78 ms
Wall time: 3.79 ms


In [36]:
%%time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
cls_true = cls_test = generator_test.classes
accuracy = accuracy_score(cls_true, cls_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(cls_true, cls_pred, average='weighted')

print("weighted")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

weighted
Precision:  0.96261014508
Recall:  0.972972972973
F1 score:  0.967234110176
CPU times: user 4.43 ms, sys: 0 ns, total: 4.43 ms
Wall time: 4.25 ms
